<a href="https://colab.research.google.com/github/dhanushpandian/RAG_Pipeline/blob/main/zephyr-7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai


In [ ]:
!pip install langchain sentence-transformers chromadb

In [41]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Chroma

from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

In [24]:
import os
from getpass import getpass
HF_token=getpass()

··········


In [74]:
os.environ['HUGGINGFACE_API_TOKEN']=HF_token

In [43]:
URL="https://dhanushpandian.co/"


In [44]:
data=WebBaseLoader(URL)

In [45]:
content=data.load()
print(content)

[Document(page_content="\n\n\n\n\n\nDhanush D - Portfolio\n\n\n\n\nContact\nAbout Me\nSkills\nEducation\nInternships\nProjects\nAchievements\nCertifications\n\n\n\nDhanush D\n3rd Year B-Tech IT Student\n\n\nContact Information\nPhone: +91-6381206640\nEmail: dhanushdpandian@gmail.com\nWebsite: dhanushpandian.co\nGithub: github.com/dhanushpandian\nLinkedin: linkedin.com/in/dhanush-d-pandian-481306249\nAddress: 381, 27th street, 6th sector, K.K.Nagar, Chennai - 600078.\n\n\nAbout Me\nAn Enthusiastic and dedicated pre-final year Information Technology student with a passion for technology and with a diverse skill set. Eager to contribute my knowledge and skills to dynamic projects while continuously expanding my understanding of emerging technologies.\nLanguages Known\nTamil, English, Hindi, Telugu\nPersonality Traits\nProblem Solving, Critical Thinking, Idea Visualization, Man-Management, Negotiation, Leadership.\nHobbies\nField Hockey (represented various institutions and clubs in state 

In [69]:
#chunking
text_splitter=RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=0)
chunking=text_splitter.split_documents(content)

In [75]:
embeddings=HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_token,
    model_name='BAAI/bge-base-en-v1.5'
)

In [71]:
vectorstore=Chroma.from_documents(chunking,embeddings)

In [79]:
model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
                      huggingfacehub_api_token=HF_token,
                     model_kwargs={"temperature":0.5,"max_new_tokens":512,"max_length":64})



In [86]:
retriver=vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":1})

In [87]:
query="contact"
docs_rel=retriver.get_relevant_documents(query)
print(docs_rel)

[Document(page_content='Contact Information\nPhone: +91-6381206640\nEmail: dhanushdpandian@gmail.com\nWebsite: dhanushpandian.co\nGithub: github.com/dhanushpandian\nLinkedin: linkedin.com/in/dhanush-d-pandian-481306249', metadata={'language': 'en', 'source': 'https://dhanushpandian.co/', 'title': 'Dhanush D - Portfolio'})]


In [89]:
qa=RetrievalQA.from_chain_type(llm=model,
                               retriever=retriver,
                               chain_type="refine")

In [100]:
q="list the projects dhanush has worked on?"

In [101]:
prompt=f"""
<|system|>
You are a AI assistant that follows instructions extremely well , Please be truthful and give direct answers
</s>
<|user|>
{q}
</s>
<|assistant|>
"""

In [102]:
response=qa(prompt)

In [103]:
print(response['result'])

Context information is below. 
------------
Dhanush D - Portfolio




Contact
About Me
Skills
Education
Internships
Projects
Achievements
Certifications



Dhanush D
3rd Year B-Tech IT Student
------------
Given the context information and not prior knowledge, answer the question: 
<|system|>
You are a AI assistant that follows instructions extremely well , Please be truthful and give direct answers
</s>
<|user|>
list the projects dhanush has worked on?
</s>
<|assistant|>

I do not have access to the specific projects worked on by dhanush d. however, according to his portfolio, he has worked on projects such as:

1. weather forecasting system using python and flask
2. sentiment analysis system using python and tensorflow
3. e-commerce website using django and postgresql
4. stock price prediction system using python and keras
5. web scraping tool using python and scrapy
6. social media analytics tool using python and tweepy

these are just a few examples, and there may be other projects